# Linear basis functions and kernels

## Introduction

In this PC-lab, we will have a look at two particular strategies in order to be able to analyze non-linear datasets, which are called _linear basis functions expansion_ and _kernels_. 

We will first have a look at linear basis functions. Next we will turn a logistic regression model to a kernel logistic regression model ourselves. Finally we will study kernel ridge regression on a real-world dataset, where we will predict the compressive strength of concrete based on its composition and age. 

## Linear basis functions

When trying to solve a binary classification problem, classifiers usually try to separate the positive from the negative class. Often, linear models are not capable of making this separation. An option to deal with this problem is to find an (extended) basis in which the classes are (almost) linearly separable. 

As an example, consider the binary classification problem in the figure below. The data depicted in this figure originates from a(n) (unknown) joint distribution, for which the Bayes-optimal decision boundary (in case of the 0/1 loss) is a circle. It is clear that the classes are not linearly separable in the original feature space (which is $\mathbb{R}^2$). An option is to extend this space using a set of non-linear functions $\phi_1 (
x_1,x_2), ... , \phi_m (x_1,x_2)$ (note that the identity map can also be chosen for $\phi_i$). Using these transformations, the representation of an instance $\mathbf{x_i} = (x_{i,1},x_{i,2})$ becomes $\mathbf{x_i} = (x_{i,1},x_{i,2}) \rightarrow \phi_1(x_{i,1},x_{i,2}),..., \phi_m(x_{i,1},x_{i,2})$. 

<div class="alert alert-success">

<b>EXERCISE 1 (warm-up): 
<br>
a) Generate a dataset according to the code that is given. Have a look at the documentation [here](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html). 
<br>
b) Which features would you construct in order to separate these two classes? 
<br>
c) Use logistic regression in order to make sure your features are working (make sure you evaluate the performance on a separate test set). In other words, fit a logistic regression model on both the original features and your new features. What's the difference in accuracy between the two? </b>
</div>

In [ ]:
##1a: 
import numpy as np
from sklearn.datasets import make_circles
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display, HTML

%matplotlib inline

plt.style.use('seaborn-white')

In [ ]:
circles_train, labels_train = make_circles(n_samples=400, noise=0.1, factor=0.6, random_state=23)
circles_test, labels_test = make_circles(n_samples=400, noise=0.1, factor=0.6, random_state=85)
blues = labels_train == 0
reds = labels_train == 1

fig, ax = plt.subplots()
ax.scatter(circles_train[reds,0],circles_train[reds,1], c='r');
ax.scatter(circles_train[blues,0],circles_train[blues,1], c='b');
ax.set_xlabel(r'$X_1$', size=18)
ax.set_ylabel(r'$X_2$', size=18)
plt.show()

In [ ]:
##1b: 
circles_train_new = circles_train[:,0]**2 + circles_train[:,1]**2
circles_test_new = circles_test[:,0]**2 + circles_test[:,1]**2

In [ ]:
##1c: 
mesh_x1, mesh_x2 = np.meshgrid(np.linspace(-2,2,100),np.linspace(-2,2,100))
grid = np.asarray([[x1,x2] for x1,x2 in zip(mesh_x1.ravel(), mesh_x2.ravel())])

lr = LogisticRegression(penalty='l2')
lr.fit(circles_train,labels_train)
pred_before = lr.predict(circles_test)
Z = lr.predict(grid)
Z = Z.reshape(mesh_x1.shape)
acc_before = accuracy_score(labels_train, pred_before)

fig, ax = plt.subplots()
ax.contourf(mesh_x1, mesh_x2, Z, cmap=plt.cm.Paired)
ax.scatter(circles_train[reds,0],circles_train[reds,1], c='red');
ax.scatter(circles_train[blues,0],circles_train[blues,1], c='blue');
ax.set_xlabel(r'$X_1$', size=18)
ax.set_ylabel(r'$X_2$', size=18)
plt.show()
print('Accuracy original features: ' + str(acc_before))

lr.fit(circles_train_new.reshape(-1,1),labels_train)
pred_after = lr.predict(circles_test_new.reshape(-1,1))
grid_new = grid[:,0]**2 + grid[:,1]**2
Z = lr.predict(grid_new.reshape(-1,1))
Z = Z.reshape(mesh_x1.shape)
acc_after = accuracy_score(labels_test, pred_after)
fig, ax = plt.subplots()
ax.contourf(mesh_x1, mesh_x2, Z, cmap=plt.cm.Paired)
ax.scatter(circles_train[reds,0],circles_train[reds,1], c='red');
ax.scatter(circles_train[blues,0],circles_train[blues,1], c='blue');
ax.set_xlabel(r'$X_1$', size=18)
ax.set_ylabel(r'$X_2$', size=18)
plt.show()
print('Accuracy expanded features: ' + str(acc_after))

# Kernels

Other methods exist to perform classification or regression tasks in an extended space. A general approach of doing so, is using so-called _kernels_. By using a computational shortcut, which is called the _kernel trick_, an explicit calculation of this extended space is not needed, and using a dot-product between two input vectors $\mathbf{x_i}$ and $\mathbf{x_j}$ suffices. The most simple kernel is the _linear kernel_, for which the kernel function looks as follows: 

\begin{equation}
k(\mathbf{x_i},\mathbf{x_j}) = \mathbf{x_i}^T\mathbf{x_j}. 
\end{equation}

Consider the following polynomial kernel function given by: 
\begin{equation}
k(\mathbf{x_i},\mathbf{x_j}) = (\mathbf{x_i}^T\mathbf{x_j})^2. 
\end{equation}

Let again be $\mathbf{x_i} = (x_{i,1},x_{i,2})$. Then we can give a simple illustration of the kernel trick by writing: 


\begin{align}
k(\mathbf{x_i},\mathbf{x_j}) &= (\mathbf{x_i}^T\mathbf{x_j})^2 = (x_{i,1}x_{j,1} + x_{i,2}x_{j,2})^2 \\
        &= x_{i,1}^2x_{j,1}^2 + x_{i,2}^2x_{j,2}^2 + 2x_{i,1}x_{j,1}x_{i,2}x_{j,2} \\
        &= (x_{i,1}^2, x_{i,2}^2, \sqrt{2} x_{i,1}x_{i,2})(x_{j,1}^2, x_{j,2}^2, \sqrt{2} x_{j,1}x_{j,2}) \\
        &= \phi(\mathbf{x_i})^T\phi(\mathbf{x_j}). 
\end{align}

This shows that the squared term of the dot product of vectors $\mathbf{x_i}$ and $\mathbf{x_j}$ is equivalent to the the product of the explicit feature mapping $\mathbf{\phi(x_i)}$ and $\mathbf{\phi(x_j)}$. 

Another popular kernel is the _radial basis functions (rbf)_ kernel. The feature space implied by this kernel is infinite-dimensional: 
\begin{align}
k(\mathbf{x_i},\mathbf{x_j}) = \exp\left(- \frac{||\mathbf{x_i}-\mathbf{x_j}||^2}{2\sigma^{2}} \right). 
\end{align}

Note that _kernel methods_ in machine learning make use of the kernel (Gramm) matrix $K$, which is defined as: 
\begin{equation}
K = 
\begin{bmatrix}
    k(\mathbf{x_1},\mathbf{x_1}) & ... & k(\mathbf{x_1},\mathbf{x_n}) \\
    \vdots & & \vdots \\
    k(\mathbf{x_n},\mathbf{x_1}) & ... & k(\mathbf{x_n},\mathbf{x_n}) \\
\end{bmatrix}, 
\end{equation}

where we have $n$ training instances at our disposal. Once the model is fit, it can be
used to predict the labels of new instances uses a kernel matrix of the following form, for which test instances are denoted with a $*$: 
\begin{equation}
K = 
\begin{bmatrix}
    k(\mathbf{x_1}^*,\mathbf{x_1}) & ... & k(\mathbf{x_1}^*,\mathbf{x_n}) \\
    \vdots & & \vdots \\
    k(\mathbf{x_L}^*,\mathbf{x_1}) & ... & k(\mathbf{x_L}^*,\mathbf{x_n}) \\
\end{bmatrix}, 
\end{equation}

where we now have $L$ test instances. Using a kernel gives rise to a whole scala of machine learning methods: 
- Support Vector Machines (previous lecture)
- Kernel Principal Component Analysis
- Kernel Logistic Regression (this lecture)
- Kernel Ridge Regression (this lecture)
- ....


<div class="alert alert-success">

<b>EXERCISE 2: 
<br>
a) Implement the rbf-kernel and calculate the kernel matrix for the previous dataset. In other words, expand the dataset from the first exercise to the new 'kernel' space. (Tip: You can do this yourself, or use your beloved machine learning library). 
<br>
b) Fit a logistic regression model to the expanded features, using the kernel matrix. In this way, we are building our own kernel logistic regression model. What is the performance on the test set?
<br>
c) Which other kernels might work for this dataset? 
 </b>
</div>

In [ ]:
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import rbf_kernel
K_train = rbf_kernel(circles_train,circles_train)
print(K_train.shape)
K_test = rbf_kernel(circles_test,circles_train)
print(K_test.shape)
lr = LogisticRegression(penalty='l2')
lr.fit(K_train,labels_train)
pred_kernel = lr.predict(K_test)
acc_kernel = accuracy_score(labels_test, pred_kernel)
print(acc_kernel)

Example for rbf-kernel matrix, which you calculate yourself: 

In [ ]:
def rbfKernel(X1, X2, sigma=0.5):
    """(Pre)calculates kernel Matrix K"""

    K_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            #Make sure vectors are column vectors: 
            x1 = x1.flatten()
            x2 = x2.flatten()
            K_matrix[i, j] = np.exp(- np.sum( np.power((x1 - x2),2) ) / float(2*(sigma**2) ) )
    return K_matrix

K_train = rbfKernel(circles_train,circles_train)
K_test = rbfKernel(circles_test,circles_train)
lr.fit(K_train,labels_train)
pred_kernel = lr.predict(K_test)
acc_kernel = accuracy_score(labels_test, pred_kernel)
print(acc_kernel)

## Kernel Ridge Regression

As stated, lots of methods make use of kernels. Another example is ridge regression, which can be extended to kernel ridge regression (KRR) in quite a standard way. By making use of the kernel trick, the method will fit a linear model (using an $l_2$-penalty) in the expanded kernel space. 

We will now have a look at the 'concreteComprStrength.txt' dataset. This dataset can be used to predict the compressive strength of concrete based on its composition. Have a look at the file 'concreteComprStrength.info' for more information concerning the variables. The target variable to predict is the 'Concrete compressive strength'.  

<div class="alert alert-success">

<b>EXERCISE 3: 
<br>
a) Look at the implementation of ridge regression and kernel ridge regression. Implement both of them to analyze the `concrete` dataset. For KRR, try both the polynomial and rbf-kernel. Which one works better? Evaluate your performance on a 30% held-out test set in terms of the mean squared error (MSE). 
<br>
b) KRR can be tweaked in various ways. How many hyperparameters do you have? Choose the polynomial kernel and start doing this by hand to get a feel of the hyperparameters. What's the influence of each hyperparameter? Can you visualize this? 
<br> 
 </b>
</div>

In [ ]:
##3a: 
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

!wget https://raw.githubusercontent.com/tfmortie/teaching/main/Kernels/concreteComprStrength.txt
df = pd.read_table('concreteComprStrength.txt', delim_whitespace=True, header=0, index_col=None)
df = df.sample(frac=1)
features = ['cement', 'blastFurnaceSlag', 'flyAsh', 'water', 'superelastizer', 'coarseAggregate', 'fineAggregate', 'age']
target = ['compressiveStrength']

X_train, X_test, y_train, y_test = train_test_split(df.loc[:,features], df.loc[:,target], test_size=0.30, random_state=85)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

rr = RidgeCV()
rr.fit(X_train_stand,y_train)
y_pred = rr.predict(X_test_stand)
mse = mean_squared_error(y_test,y_pred)
print('MSE Ridge Regression: ' + str(mse))

krr = KernelRidge(alpha=1, kernel='poly', gamma=0.1, degree=5)
krr.fit(X_train_stand,y_train)
y_pred = krr.predict(X_test_stand)
mse = mean_squared_error(y_test,y_pred)
print('MSE Kernel Ridge Regression using polynomial kernel: ' + str(mse))

krr = KernelRidge(alpha=1, kernel='rbf', gamma=0.1)
krr.fit(X_train_stand,y_train)
y_pred = krr.predict(X_test_stand)
mse = mean_squared_error(y_test,y_pred)
print('MSE Kernel Ridge Regression using rbf kernel: ' + str(mse))



In [ ]:
##3b: 
alphas = np.logspace(-3,2,50)
mse_alphas = np.zeros(len(alphas))
i=0
for alpha in alphas: 
    krr = KernelRidge(alpha=alpha, kernel='poly', gamma=1, degree=2)
    krr.fit(X_train_stand,y_train)
    y_pred = krr.predict(X_test_stand)
    mse_alphas[i] = mean_squared_error(y_test,y_pred)
    i+=1

fig, ax = plt.subplots()
ax.scatter(alphas,mse_alphas, c='r');
ax.set_xlabel(r'$\lambda$', size=18)
ax.set_ylabel('MSE', size=18)
ax.set_xscale('log')
ax.set_xlim(0.001,100)
plt.show()

gammas = np.logspace(-3,2,50)
mse_gammas = np.zeros(len(gammas))
i=0
for gamma in gammas: 
    krr = KernelRidge(alpha=1, kernel='poly', gamma=gamma, degree=2)
    krr.fit(X_train_stand,y_train)
    y_pred = krr.predict(X_test_stand)
    mse_gammas[i] = mean_squared_error(y_test,y_pred)
    i+=1

fig, ax = plt.subplots()
ax.scatter(gammas,mse_gammas, c='r');
ax.set_xlabel(r'$\gamma$', size=18)
ax.set_ylabel('MSE', size=18)
ax.set_xscale('log')
ax.set_xlim(0.001,100)
plt.show()

degrees = np.arange(1,10)
mse_degrees = np.zeros(len(degrees))
i=0
for degree in degrees: 
    krr = KernelRidge(alpha=1, kernel='poly', gamma=0.1, degree=degree)
    krr.fit(X_train_stand,y_train)
    y_pred = krr.predict(X_test_stand)
    mse_degrees[i] = mean_squared_error(y_test,y_pred)
    i+=1

fig, ax = plt.subplots()
ax.scatter(degrees,mse_degrees, c='r');
ax.set_xlabel('Degree', size=18)
ax.set_ylabel('MSE', size=18)
ax.set_xlim(0,12)
plt.show()